In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier

In [2]:
train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)
sample = pd.read_csv('submit_sample.csv', header=None)

In [3]:
test['y'] = -999

In [4]:
all_df = pd.concat([train, test], axis=0)
all_df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
id,,,,,,,,,,,,,,,,,
0,31,services,married,secondary,no,12294,yes,no,cellular,21,nov,101,3,498,0,other,0
1,29,entrepreneur,single,tertiary,no,43027,no,no,cellular,22,aug,158,2,702,0,unknown,1
2,35,management,married,tertiary,no,12252,yes,no,cellular,11,nov,351,1,826,0,failure,0
3,31,technician,married,secondary,no,99121,yes,yes,unknown,16,may,658,2,120,0,failure,0
4,48,unemployed,married,primary,no,42005,yes,no,telephone,3,apr,177,1,273,0,unknown,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18045,49,self-employed,married,tertiary,no,98357,yes,no,cellular,6,jul,101,2,417,0,failure,-999
18046,34,blue-collar,married,secondary,no,29621,yes,no,cellular,12,may,345,1,815,0,unknown,-999
18047,34,admin.,single,secondary,no,94260,yes,no,unknown,16,may,121,2,370,0,unknown,-999


In [5]:
import category_encoders as ce
ce_ohe = ce.OneHotEncoder(handle_unknown='impute')
ce_ohe.fit(all_df)

OneHotEncoder(cols=['job', 'marital', 'education', 'default', 'housing', 'loan',
                    'contact', 'month', 'poutcome'],
              handle_unknown='impute')

In [6]:
all_df = ce_ohe.transform(all_df)

In [7]:
all_df

,age,job_1,job_2,job_3,job_4,job_5,job_6,job_7,job_8,job_9,...,month_11,duration,campaign,pdays,previous,poutcome_1,poutcome_2,poutcome_3,poutcome_4,y
id,,,,,,,,,,,,,,,,,,,,,
0,31,1,0,0,0,0,0,0,0,0,...,0,101,3,498,0,1,0,0,0,0
1,29,0,1,0,0,0,0,0,0,0,...,0,158,2,702,0,0,1,0,0,1
2,35,0,0,1,0,0,0,0,0,0,...,0,351,1,826,0,0,0,1,0,0
3,31,0,0,0,1,0,0,0,0,0,...,0,658,2,120,0,0,0,1,0,0
4,48,0,0,0,0,1,0,0,0,0,...,0,177,1,273,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18045,49,0,0,0,0,0,0,0,0,1,...,0,101,2,417,0,0,0,1,0,-999
18046,34,0,0,0,0,0,1,0,0,0,...,0,345,1,815,0,0,1,0,0,-999
18047,34,0,0,0,0,0,0,1,0,0,...,0,121,2,370,0,0,1,0,0,-999


In [8]:
X = all_df[all_df['y'] != -999].drop('y', axis=1)
y = all_df[all_df['y'] != -999]['y']
X.shape, y.shape

((27100, 50), (27100,))

In [28]:
parameters = {
#     'max_depth': list(range(2, 11)),
    'max_depth': [5],
#     'min_samples_leaf': [5,10,20,50,100,500],
    'min_samples_leaf': [5],
#     'objective' : ['binary', 'cross_entropy'],
    'learning_rate': [0.1],
#     'learning_rate': [0.005, 0.05, 0.5, 0.1],
    'random_state': [0],
#     'n_estimators': [1000],
    'n_estimators': [40, 100, 1000],
#     'num_leaves': [6,8,12,16],
#     'early_stopping_round': [10],
}

In [29]:
lgb = LGBMClassifier()

gcv = GridSearchCV(lgb, parameters, cv=5, scoring="roc_auc", n_jobs=-1, return_train_score=True)
gcv.fit(X, y)


GridSearchCV(cv=5, estimator=LGBMClassifier(), n_jobs=-1,
             param_grid={'learning_rate': [0.1], 'max_depth': [5],
                         'min_samples_leaf': [5],
                         'n_estimators': [40, 100, 1000], 'random_state': [0]},
             return_train_score=True, scoring='roc_auc')

In [30]:
gcv.best_params_

{'learning_rate': 0.1,
 'max_depth': 5,
 'min_samples_leaf': 5,
 'n_estimators': 100,
 'random_state': 0}

In [31]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y, gcv.predict_proba(X)[:,1])
# 0.8294283339336216

# {'learning_rate': 0.1,
#  'max_depth': 5,
#  'min_samples_leaf': 5,
#  'objective': 'binary',
#  'random_state': 0}
# 0.9113408811229391

# {'learning_rate': 0.1,
#  'max_depth': 5,
#  'min_samples_leaf': 5,
#  'num_leaves': 12,
#  'objective': 'binary',
#  'random_state': 0}
# 0.8881623595992743


0.9113408811229391

In [17]:
gcv2 = GridSearchCV(lgb, parameters, cv=5, n_jobs=-1, return_train_score=True)
gcv2.fit(X, y)


GridSearchCV(cv=5, estimator=LGBMClassifier(), n_jobs=-1,
             param_grid={'learning_rate': [0.005, 0.05, 0.5, 0.1],
                         'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'min_samples_leaf': [5, 10, 20, 50, 100, 500],
                         'num_leaves': [6, 8, 12, 16],
                         'objective': ['binary', 'cross_entropy'],
                         'random_state': [0]},
             return_train_score=True)

In [18]:
gcv2.best_params_

{'learning_rate': 0.1,
 'max_depth': 5,
 'min_samples_leaf': 5,
 'num_leaves': 16,
 'objective': 'binary',
 'random_state': 0}

In [19]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y, gcv2.predict_proba(X)[:,1])

# {'learning_rate': 0.05,
#  'max_depth': 7,
#  'min_samples_leaf': 5,
#  'objective': 'binary',
#  'random_state': 0}
# 0.9001029206140159


0.89804914123931